In [1]:
import sys
import os
import numpy as np
from gensim.models import KeyedVectors
from text.encoder import TextEncoder
from text.summarizer import NewsSummarizer
from common.space import Space
from config import w2v_file, vocab_limit
from config import get_logger

sys.path.append(os.path.join(sys.path[0],'image','captioning')) # add models.py

logger = get_logger('recommender')

In [2]:
keyed_vectors = KeyedVectors.load_word2vec_format(w2v_file, limit=vocab_limit, binary=True)
space = Space(keyed_vectors)

2020-02-17 21:34:44,001 gensim.models.utils_any2vec: INFO loading projection weights from ~/Downloads/GoogleNews-vectors-negative300.bin.gz
2020-02-17 21:35:29,943 gensim.models.utils_any2vec: INFO loaded (1000000, 300) matrix from ~/Downloads/GoogleNews-vectors-negative300.bin.gz


In [3]:
class Recommender:

    def __init__(self, space: Space):
        self.space = space
        self.text_encoder = TextEncoder(space=self.space)
        self.image_subspaces_loc = None
    
    def set_image_subspaces(self, path: str):
        self.image_subspaces_loc = path
                
    def predict(self, text: str, count: int = 5):
        sims = self.compute_similarities(text)
        return sorted(sims, key=lambda x: x[1], reverse=True)[:count]

#     def display_predictions(self, count: int=5):
#         preds = self.predict(text, count)
#         pass

    # Search the most similar word subspace from images
    def compute_similarities(self, text: str):
        sub_txt = self.text_encoder.create_subspace(full_text=text)
        logger.info(sub_txt.shape)
        sims = []
        if self.image_subspaces_loc is not None:
            for im in os.listdir(self.image_subspaces_loc):
                sub_img = np.load(os.path.join(self.image_subspaces_loc, im))
                sim = self.space.subspaces_similarity(sub_txt, sub_img)
                sims.append((im.split('.')[0], sim))
        return sims
        
    def __str__(self):
        return "Image Recommender Model"

In [4]:
goi_subspaces = '../data/subspaces/goi5k'

In [5]:
recommender = Recommender(space=space)
recommender.set_image_subspaces(goi_subspaces)

In [6]:
text = """
In Anaheim, California, thousands of cosplayers came to WonderCon wearing their costumes and make-up and brought props. People of all ages come to the annual event to see their favourite fandom in games, comics, TV shows, movies, books, and costumes!"""

In [7]:
recommender.text_encoder._get_keywords(text)

Counter({'costume': 2,
         'annual': 1,
         'tv': 1,
         'event': 1,
         'game': 1,
         'fandom': 1,
         'comics': 1})

In [10]:
text = """
Shocking CCTV footage released by Manchester police shows the moment the man wielding 
a large-bladed knife is tackled to the ground by armed officers. At about 11 pm on Tuesday, 
CCTV operators spotted a man waving the butcher’s knife around the Piccadilly Garden’s 
area of Manchester and informed the police. The man can be seen struggling to stand and 
interacts with terrified members of the public, as he continues to wave the knife around.
A 55-year-old man has been arrested on suspicion of affray and remains in police custody 
for questioning."""
text2 = """
a close up of a green head on a table. a close up of a green head on a table.
"""

In [16]:
# recommender.compute_similarities(text=text)
recommender.predict(text=text)

2020-01-30 13:31:59,461 recommender: INFO (300, 5)


[('14372026358_a2ce9435fb_o', 0.14994904),
 ('6654575639_e87751962c_o', 0.14353785),
 ('3241537537_f5c850b93b_o', 0.14353785),
 ('14785988292_7e617261e2_o', 0.13940585),
 ('3830496092_cd80a8353d_o', 0.13847227)]

In [8]:
from image.encoder import ImageEncoder
from config import google_open_images_folder
image_encoder = ImageEncoder(space=space)

In [9]:
def test_subspace_from_image(image_filename: str):
    captions: list = image_encoder._get_captions(image_filename)
    print(captions)
    keywords: Counter = image_encoder._get_keywords(captions)
    print(keywords)
    word_subspace = image_encoder.create_subspace(image_filename)
    return word_subspace

In [10]:
image_filename = os.path.join(google_open_images_folder, '3361086680_baf1ebe647_o.jpg')
ws = test_subspace_from_image(image_filename)

['a close up of a broccoli head on a table', 'a close up of a broccoli head on a table', 'a close up of a piece of broccoli', 'a close up of a piece of broccoli', 'a close up of a piece of broccoli']
Counter({'piece': 1, 'head': 1, 'table': 1, 'broccoli': 1})
